In [7]:
import pandas as pd
import math
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor
import re


In [31]:
df = pd.read_csv('Data/yad2_test.csv',index_col=0)
df2 = pd.read_csv("Data/Nadlan_clean.csv",index_col=0)

df = df.drop(columns=['Balconies','Parking','Balconies','asset_classification','immediate'
                      ,'ac','Furniture','on_pillars','elevator','storeroom','Shelter','home_number_2',
                     'text','date_of_entry','images','Handicapped','Floors_text'], axis=1)




def update_neighborhood_street(df):
    t = "'"
    df['Street'] = df['Street'].str.replace(t,"")
    df['Street'] = df['Street'].str.replace("כרמייה","כרמיה")


    df['Neighborhood'] = df['Neighborhood'].str.replace(t,"")
    df['Neighborhood'] = df['Neighborhood'].str.replace("נווה חן","נוה חן")


    return df
df = update_neighborhood_street(df)

def edit_distance(str1, str2):
    '''
     The function returns the absolute value of the number of 
     characters that need to be added/changed/deleted to get from the first string to the second.
     Dynamic programming algorithm to compute the edit distance between two strings. O(mn)
    '''
    m = len(str1)
    n = len(str2)
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)]
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i][j-1], dp[i-1][j], dp[i-1][j-1])
    return dp[m][n]
 
# for s1 in street1:
#     for s2 in street2:
#         dis = edit_distance(s1,s2)
#         if dis == 1:
#             print(f"s1: {s1} s2: {s2}\n")
            
def unifrom_neighborhood(df):
    df2 = pd.read_csv("Data/Nadlan_clean.csv",index_col=0)

    df =df.dropna(subset =['Street', 'Neighborhood'] )
    df2 =df2.dropna(subset =['Street', 'Neighborhood'] )
    nb1 = df['Neighborhood'].unique()
    nb2 = df2['Neighborhood'].unique()
    for n1 in nb1:
        for n2 in nb2:

            dis = edit_distance(n1,n2)
            if dis ==1:
                print(f"s1: {n1} s2: {n2}\n")
                
# unifrom_neighborhood(df)
def convert_price(df):
    df['Price'] = df['Price'].str.replace('₪', '')
    df['Price'] = df['Price'].str.replace(',', '')
    df['Price'] = df['Price'].astype(np.int32)
    

    return df

    return df
t = df[df['Size'] == 440]
t
# df = convert_price(df)
# df = df.dropna('Price')
# df = df.sort_values('Price')
# df

,Price,Street,Neighborhood,Size,Floors,street_id,Rooms,Floor,Lat,Long,Home_number
462,"59,000,000 ₪",הירקון,"לב תל אביב, לב העיר צפון",440.0,6.0,461.0,8.0,6.0,32.077098,0.0,NaN


In [9]:
def convert_coordinates(lat_long_list):
    '''
    This function use to convert UTM coordinates to ITM coordinates using outside website with selenium 
    '''
    driver = webdriver.Chrome()
    driver.get("https://zvikabenhaim.appspot.com/software/ITM/")

    # locate all the input fields and button
    lat_input = driver.find_element(By.XPATH, '//*[@id="lat"]')
    lon_input = driver.find_element(By.XPATH, '//*[@id="long"]')
    convert_button = driver.find_element(By.XPATH, '//form[1]/table/tbody/tr[3]/td/input')
    east_span = driver.find_element(By.XPATH, '//*[@id="itm_east"]')
    north_span = driver.find_element(By.XPATH, '//*[@id="itm_north"]')
    
    
    coverted_list = []
    for cords in lat_long_list:
        east = cords[0]
        north = cords[1]
        if cords[0] != 0.0 and cords[1] != 0.0:
            lat_input.clear()
            lon_input.clear()

            # send lat and long to input fields

            lat_input.send_keys(cords[0])
            lon_input.send_keys(cords[1])

            convert_button.click()


            east = east_span.text
            north = north_span.text
            
            
        coverted_list.append((east,north))
        

    driver.quit()
    return coverted_list

def convert_lat_long(df):

    def get_long_lat_tuples(df):
        long_lat_tuples = []
        for index, row in df.iterrows():
            lat = row['Lat']
            long = row['Long']

            try:
                lat = float(lat)
            except:
                lat = 0.0
            try:
                long = float(long)
            except:
                long = 0.0
                
            long_lat_tuples.append((lat, long))


        return long_lat_tuples
    
    new_cord = convert_coordinates(get_long_lat_tuples(df))
    
    for index, row in df.iterrows():  
        try:
            df.at[index,'Lat'] = new_cord[index][0]
            df.at[index,'Long'] = new_cord[index][1]
        except:
            pass


    return df



In [10]:
def find_avg_cords_by_street():
    df = pd.read_csv("Data/Nadlan_clean.csv",index_col=0)
    df = df.dropna(subset=['Long','Lat','Street']).reset_index(drop=True)
    df['Long'] = df['Long'].astype(int)
    df['Lat'] = df['Lat'].astype(int)
    
    # create unique streets list
    streets = df['Street'].unique()
    
    street_cords_avg = {}
    for street in streets:
        street_df = df[df['Street'] == street]
        
        lat_mean = street_df['Lat'].mean()
        long_mean = street_df['Long'].mean()

        street_cords_avg[street] = (long_mean ,lat_mean)
        
    return street_cords_avg
        
def complete_long_lat_columns(df,street_cords_avg):
    for index, row in df.iterrows():
        long = row['Long']
        lat = row['Lat']
        street = row['Street']
        
        try:
            cords = street_cords_avg[street]         
            if long == 0.0 or lat == 0.0:
                df.at[index, 'Long'] = cords[1]
                df.at[index, 'Lat'] = cords[0]

                
        except:
            pass

    
    df.rename(columns = {'Lat':'Temp'}, inplace = True)
    df.rename(columns = {'Long':'Lat'}, inplace = True)
    df.rename(columns = {'Temp':'Long'}, inplace = True)

    df = df.loc[(df['Lat'] != 0.0) & (df['Long'] != 0.0)]
    
    df['Long'] = df['Long'].astype(np.int32)
    df['Lat'] = df['Lat'].astype(np.int32)
    
    return df
            
df = convert_lat_long(df)
df =complete_long_lat_columns(df,find_avg_cords_by_street())
df

<ipython-input-10-a52e4d84798f>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Long'] = df['Long'].astype(np.int32)
<ipython-input-10-a52e4d84798f>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Lat'] = df['Lat'].astype(np.int32)


,Price,Street,Neighborhood,Size,Floors,street_id,Rooms,Floor,Long,Lat,Home_number
0,"7,300,000 ₪",הדולפין,"עגמי, גבעת העליה",130.0,4.0,1753.0,5.0,2.0,176568,661898,NaN
1,"6,190,000 ₪",אבן סינא,"עגמי, גבעת העליה",170.0,1.0,1713.0,7.0,0.0,176509,661656,NaN
2,"8,800,000 ₪",בזל,הצפון הישן - צפון,120.0,20.0,450.0,4.0,20.0,179325,666369,NaN
6,"1,470,000 ₪","לח""י","התקוה, בית יעקב, נווה צה""ל",50.0,4.0,2860.0,3.0,2.0,180797,661678,NaN
7,"2,680,000 ₪",וינגייט,יד אליהו,65.0,30.0,2639.0,2.0,15.0,180368,663026,NaN
...,...,...,...,...,...,...,...,...,...,...,...
926,"4,300,000 ₪",הירקון,כרם התימנים,70.0,4.0,461.0,3.0,1.0,178361,665493,NaN
929,"7,190,000 ₪",סוטין,הצפון החדש - דרום,90.0,27.0,1021.0,3.0,15.0,180011,665485,29.0
930,"2,950,000 ₪",מעון,פלורנטין,50.0,9.0,1602.0,2.0,1.0,178116,662572,NaN
931,"6,750,000 ₪",קדם,"עגמי, גבעת העליה",170.0,4.0,1760.0,5.0,1.0,176318,661066,NaN


In [11]:
df = df.dropna(subset=['Street']).reset_index(drop=True)
print(df.shape)

(453, 11)


In [12]:
def add_neighborhood():
    df = pd.read_csv("Data/Real_Estate_TLV_GOVMAPS_1.csv",index_col=0)
    neighborhoods = {}
    for index, row in df.iterrows():
        neighborhood = row['NEIGHBORHOOD']
        street = row['STREENNAME']
        if neighborhood not in neighborhoods:
            neighborhoods[neighborhood] = set()
        neighborhoods[neighborhood].add(street)
    return neighborhoods

def complete_neighborhood_column(df, neighborhoods):
    # Loop through each row in the DataFrame
    for index, row in df.iterrows():
        street = row['Street']
        
        # Loop through each neighborhood in the neighborhoods dictionary
        for neighborhood, streets in neighborhoods.items():
            # Check if the street is in the set of streets for the neighborhood
            if street in streets:
                # If it is, assign the neighborhood to the 'neighborhood' column for this row
                df.at[index, 'Neighborhood'] = neighborhood
                break # Break out of the inner loop once we find a match
    
    return df

df = complete_neighborhood_column(df,add_neighborhood())

In [13]:
def create_building_year_list():
    df = pd.read_csv("Data/Nadlan_clean.csv",index_col=0)
    df = df.dropna(subset=['Floors','Street','Build_year']).reset_index(drop=True)
    
    add_build_year = []
    df['Floors'] = df['Floors'].astype(int)
    df['Street'] = df['Street'].astype(str)
    df['Build_year'] = df['Build_year'].astype(int)
    
    for index, row in df.iterrows():
        floors = row['Floors']
        street = row['Street']
        build_year = row['Build_year']
        
        build_string = (str(floors)+str(street)+str(build_year)).strip()
        
        add_build_year.append(build_string)
        
    # remove duplitcates 
    return list(set(add_build_year))

def complete_build_year_column(df, add_build_year):
    df['Build_year'] = 0
    df['Floors'] = df['Floors'].fillna(0)
    
    df['Floors'] = df['Floors'].astype(int)
    df['Street'] = df['Street'].astype(str)
    
    # Loop through each row in the DataFrame
    for index, row in df.iterrows():
        floors = row['Floors']
        street = row['Street']
        build_year = row['Build_year']

        build_string = (str(floors)+str(street)).strip().strip()

        for item in add_build_year:
            floors_street_year = item.strip()
            
            if floors_street_year.startswith(build_string):
                year = floors_street_year[-4:]
                df.at[index,'Build_year'] = year
                break

    return df






In [14]:
def find_avg_building_old_by_street():
    df = pd.read_csv("Data/Nadlan_clean.csv",index_col=0)
    df = df.dropna(subset=['Build_year','Street']).reset_index(drop=True)
    df['Build_year'] = df['Build_year'].astype(int)
    df['Street'] = df['Street'].astype(str)
    
    # create unique streets list
    streets = df['Street'].unique()
    
    street_buildings_avg = {}
    for street in streets:
        
        street_df = df[df['Street'] == street]
        street_mean = street_df['Build_year'].mean()
        
        
        street_buildings_avg[street] = street_mean
        
    return street_buildings_avg
        
def complete_building_old_by_street(df,avg_bulding_old_street):
    for index, row in df.iterrows():
        old = row['Build_year']
        street = row['Street']
        if old == 0:
            try:
                avg = avg_bulding_old_street[street] 
                df.at[index, 'Build_year'] = avg
            except:
                df.drop(index = index , inplace =True)
            
            
        
    df['Build_year'] = df['Build_year'].astype(np.int32)

    return df

df = complete_build_year_column(df,create_building_year_list())

df = complete_building_old_by_street(df,find_avg_building_old_by_street())


In [15]:
def calc_distance_from_the_see_TLV(X_coordinate, Y_coordinate):
    north_x = 180471
    north_y = 672391
    south_x = 177333
    south_y = 663016
    
    m = (south_y - north_y) / (south_x - north_x)   
    b = north_y - (m * north_x)
    
    numerator = abs(m * X_coordinate - Y_coordinate + b)
    denominator = math.sqrt(m**2 + 1)
    return numerator / denominator


def calc_distance_from_train_station(df):
    stations = [(179820.47,662424.54), (180619,664469.56), (181101.44,665688.78), (181710.96,667877.05)]
    distances = []

     # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        apartment_coords = (row['Lat'], row['Long'])

        # Calculate the distance between the apartment and each train station
        station_distances = []
        for station_coords in stations:       
            station_distance = abs(station_coords[0] - apartment_coords[1]) + abs(station_coords[1] - apartment_coords[0])
            station_distances.append(station_distance)

        # Find the minimum distance
        min_distance = min(station_distances)
        distances.append(min_distance)

    df['Train'] = distances
    df['Train'] = df['Train'].astype(np.int32)
    return df
    
def distance_from_sea_tlv(df):
    '''
    Need to fix long ? lat ? 
    '''
    df = df.dropna(subset=['Lat', 'Long']).reset_index(drop=True)
    latitudes = df['Long'].astype(float)
    longitudes = df['Lat'].astype(float)
    distances = [calc_distance_from_the_see_TLV(lat, lon) for lat, lon in zip(latitudes, longitudes)]
    df['Distance_sea'] = distances
    df['Distance_sea'] = df['Distance_sea'].astype(np.int32)
    return df

In [16]:
def neighborhood_to_numeric(df):
    # Not in use
    df.dropna(subset=['neighborhood'] ,inplace=True)
    df.reset_index(drop=True,inplace =True)
    t = df['neighborhood'].unique()
    df['neighborhood'].replace(t.tolist(),[i for i in range(len(t))], inplace=True)
    return df


def edit_parking(df):
    # Not in use
    df['parking'] = df['parking'].str.replace('ללא','0')
    df['parking'] = df['parking'].astype(int)
    return df

    
def add_date(df):
    # all the deals need to be 2023
    df['Year'] = 2023
    df.assign(profit='NAN')
    df['Year'].astype(np.int32)
    return df

def convert_price(df):
    df['Price'] = df['Price'].apply(lambda x: re.sub('[^0-9\.]','',x)).astype(np.int32)
    df['Price'] = df['Price'].apply(lambda x: re.sub(r'[^\d]', '', x)).(np.int32)

    return df


In [17]:

df = convert_price(df)
df = distance_from_sea_tlv(df)
df = calc_distance_from_train_station(df)
df = add_date(df)

# df = edit_parking(df)
# df = neighborhood_to_numeric(df)

In [18]:
df


,Price,Street,Neighborhood,Size,Floors,street_id,Rooms,Floor,Long,Lat,Home_number,Build_year,Distance_sea,Train,Year
0,7300000,הדולפין,עג'מי וגבעת עליה,130.0,4,1753.0,5.0,2.0,176568,661898,NaN,1910,370,3779,2023
1,6190000,אבן סינא,עג'מי וגבעת עליה,170.0,1,1713.0,7.0,0.0,176509,661656,NaN,2006,349,4080,2023
2,8800000,בזל,הצפון הישן-החלק הצפוני,120.0,20,450.0,4.0,20.0,179325,666369,NaN,1990,824,2456,2023
3,1470000,"לח""י",התקווה,50.0,4,2860.0,3.0,2.0,180797,661678,NaN,1950,3709,1723,2023
4,2680000,וינגייט,יד אליהו,65.0,30,2639.0,2.0,15.0,180368,663026,NaN,1951,2874,1148,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,4300000,הירקון,הצפון הישן-החלק הצפוני,70.0,4,461.0,3.0,1.0,178361,665493,NaN,1935,188,2936,2023
427,7190000,סוטין,הצפון החדש החלק הדרומי,90.0,27,1021.0,3.0,15.0,180011,665485,29.0,1974,1755,1294,2023
428,2950000,מעון,פלורנטין,50.0,9,1602.0,2.0,1.0,178116,662572,NaN,1970,883,1851,2023
429,6750000,קדם,עג'מי וגבעת עליה,170.0,4,1760.0,5.0,1.0,176318,661066,NaN,2000,343,4861,2023


In [19]:
df.to_csv('Data/Test2.csv')

In [1]:
# import requests
# from bs4 import BeautifulSoup
# from pandas import json_normalize
# url = 'https://gisn.tel-aviv.gov.il/iview2js4/index.aspx?layers=513'
# url = 'https://gisn.tel-aviv.gov.il/arcgis/rest/services/IView2/MapServer/513/query?where=1%3D1&outFields=*&f=json'
# response = requests.get(url)
# data = response.json()
# df = json_normalize(data['features'])
# df


In [60]:
def rename_addresses_columns(df):
    df = df.drop(columns=['k_status_hesder','t_ktovet_melea','t_bayit_veknisa'], axis=1)

    df.rename(columns = {'id_ktovet':'Address_id'}, inplace = True)
    df.rename(columns = {'k_rechov':'Street_id'}, inplace = True)
    df.rename(columns = {'t_rechov':'Street'}, inplace = True)
    df.rename(columns = {'ms_bayit':'Home_number'}, inplace = True)
    df.rename(columns = {'knisa':'Entrance'}, inplace = True)
    df.rename(columns = {'x':'Long'}, inplace = True)
    df.rename(columns = {'y':'Lat'}, inplace = True)
    df.rename(columns = {'ms_gush':'Gush'}, inplace = True)
    df.rename(columns = {'ms_chelka':'Helka'}, inplace = True)
    df.rename(columns = {'UniqueId':'Full_id'}, inplace = True)
    df.rename(columns = {'lon':'Long_gis'}, inplace = True)
    df.rename(columns = {'lat':'Lat_gis'}, inplace = True)
    return df

df_a = pd.read_csv('Data/Addresses.csv')
df_a = rename_addresses_columns(df_a)
df_a
# df_a.to_csv('Data/Addresses_update.csv')

,Address_id,Street_id,Street,Home_number,Entrance,Long,Lat,Gush,Helka,Full_id,Long_gis,Lat_gis
0,89611,823,קהילת ריגא,16,,183321.38,669102.73,6625.0,567.0,527-89611,34.821559,32.114708
1,97982,2024,בארט אהרון,9,,181170.53,669101.95,6649.0,235.0,527-97982,34.798769,32.114628
2,132822,823,קהילת ריגא,14,,183310.69,669078.20,6625.0,566.0,527-132822,34.821446,32.114487
3,88629,804,רפידים,34,,182479.19,669077.53,6627.0,302.0,527-88629,34.812636,32.114452
4,94363,983,קהילת פוזנא,7,,183158.56,669059.09,6625.0,560.0,527-94363,34.819835,32.114309
...,...,...,...,...,...,...,...,...,...,...,...,...
52336,87653,778,בצרון,13,,181054.41,663753.44,6150.0,425.0,527-87653,34.797752,32.066392
52337,206196,779,עמינדב,30,א,180935.78,663555.73,6150.0,802.0,527-206196,34.796503,32.064606
52338,86336,729,ההשכלה,10,,180940.39,663696.64,6150.0,526.0,527-86336,34.796546,32.065876
52339,139580,1075,יניב,6,,181216.09,663785.40,6150.0,744.0,527-139580,34.799462,32.066686


In [16]:
def edit_distance(str1, str2):
    '''
     The function returns the absolute value of the number of 
     characters that need to be added/changed/deleted to get from the first string to the second.
     Dynamic programming algorithm to compute the edit distance between two strings. O(mn)
    '''
    m = len(str1)
    n = len(str2)
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)]
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif str1[i-1] == str2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i][j-1], dp[i-1][j], dp[i-1][j-1])
    return dp[m][n]


In [ ]:
# url = 'https://gw.yad2.co.il/latest-deals/completed?area=1&city=5000&limit=10000'
# data = requests.get(url).json()
# df3 = pd.DataFrame(data["data"])


# df3